In [24]:
# Classification with RandomForest

import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load and clean training data
train_df = pd.read_csv("extracted_features.csv")
for col in train_df.select_dtypes(include="object"):
    train_df[col] = train_df[col].str.strip("[]").astype(float)

# Train RandomForest
X_train = train_df.drop("label", axis=1)
y_train = train_df["label"]
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Load and clean new examples
new_df = pd.read_csv("extracted_features_udio.csv", index_col=0)
# drop any stray index or old label columns
drop_cols = [c for c in new_df.columns if c.startswith("Unnamed")] + ["label"]
new_df = new_df.drop(columns=[c for c in drop_cols if c in new_df], errors="ignore")
for col in new_df.select_dtypes(include="object"):
    new_df[col] = new_df[col].str.strip("[]").astype(float)
new_df["label"] = 1

# Predict and report
X_new = new_df.drop("label", axis=1)
y_new = new_df["label"]
preds = rf.predict(X_new)

print(f"Random Forest accuracy: {accuracy_score(y_new, preds):.3f}")


Random Forest accuracy: 0.255


In [17]:
# We observe very low accuracy on Udio songs with the naive RandomForest approach.
# Try a more nuanced approach: Top-10 features only, SVM, with hyperparameter tuning

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

features = [
    'spectral_flatness','mfcc_6_std','mfcc_6_mean',
    'shimmer','mfcc_4_std','mfcc_1_mean',
    'dynamic_range','jitter','mfcc_5_mean','mfcc_5_std'
]

X = train_df[features]
y = train_df['label']

pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures(2, include_bias=False)),
    ('svc', SVC(kernel='rbf'))
])

param_grid = {
    'svc__C': [0.1, 1, 10],
    'svc__gamma': ['scale', 0.1, 1]
}

grid = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid.fit(X, y)

print(f"Best CV Acc: {grid.best_score_:.3f}, Params: {grid.best_params_}")

pred = grid.predict(new_df[features])
print(f"Udio Acc: {accuracy_score(new_df['label'], pred):.3f}")


Best CV Acc: 0.957, Params: {'svc__C': 10, 'svc__gamma': 'scale'}
Udio Acc: 0.712
